In [1]:
import pandas as pd
import gender_guesser.detector as gender
import requests
import time

In [2]:
df_base = pd.read_csv("../data/imdb-movies-dataset.csv")
df_base.head(5)

,Poster,Title,Year,Certificate,Duration (min),Genre,Rating,Metascore,Director,Cast,Votes,Description,Review Count,Review Title,Review
0,https://m.media-amazon.com/images/M/MV5BYWRkZj...,The Idea of You,2023.0,R,115.0,"Comedy, Drama, Romance",6.4,67.0,Michael Showalter,"Anne Hathaway, Nicholas Galitzine, Ella Rubin,...","28,744","Solène, a 40-year-old single mom, begins an un...",166,Hypocrisy as an idea,"This film, as well as the reaction to it, is a..."
1,https://m.media-amazon.com/images/M/MV5BZGI4NT...,Kingdom of the Planet of the Apes,2023.0,PG-13,145.0,"Action, Adventure, Sci-Fi",7.3,66.0,Wes Ball,"Owen Teague, Freya Allan, Kevin Durand, Peter ...","22,248","Many years after the reign of Caesar, a young ...",183,A phenomenal start to another trilogy!,"I'm a big fan of all the planet of the apes, a..."
2,https://m.media-amazon.com/images/M/MV5BZjIyOT...,Unfrosted,2023.0,PG-13,97.0,"Biography, Comedy, History",5.5,42.0,Jerry Seinfeld,"Isaac Bae, Jerry Seinfeld, Chris Rickett, Rach...","18,401","In 1963 Michigan, business rivals Kellogg's an...",333,not funny,Pretty much the worst criticism you can lay on...
3,https://m.media-amazon.com/images/M/MV5BMjA5Zj...,The Fall Guy,2023.0,PG-13,126.0,"Action, Comedy, Drama",7.3,73.0,David Leitch,"Ryan Gosling, Emily Blunt, Aaron Taylor-Johnso...","38,953",A down-and-out stuntman must find the missing ...,384,Everything you needed and more!,Just got out of the Austin premier at SXSW and...
4,https://m.media-amazon.com/images/M/MV5BNTk1MT...,Challengers,2023.0,R,131.0,"Drama, Romance, Sport",7.7,82.0,Luca Guadagnino,"Zendaya, Mike Faist, Josh O'Connor, Darnell Ap...","32,517","Tashi, a former tennis prodigy turned coach, t...",194,"Watch ""Match Point"" instead",This is a tough one. I liked the concept and t...


In [3]:
df_base.columns = df_base.columns.str.lower().str.replace(' ', '_')
df_base = df_base[['title', 'year', 'certificate', 'genre',
       'rating', 'metascore', 'director', 'cast', 'description']]

In [4]:
df_extra = pd.read_csv("../data/IMDbMovies-Clean.csv")
df_extra.head(5)

,Title,Summary,Director,Writer,Main Genres,Motion Picture Rating,Release Year,Runtime (Minutes),Rating (Out of 10),Number of Ratings (in thousands),Budget (in millions),Gross in US & Canada (in millions),Gross worldwide (in millions),Opening Weekend in US & Canada,Gross Opening Weekend (in millions)
0,Napoleon,An epic that details the checkered rise and fa...,Ridley Scott,David Scarpa,"Action,Adventure,Biography",R,2023.0,158.0,6.7,38.0,NaN,37.514,84.968,11.26.2023,20.639
1,The Hunger Games: The Ballad of Songbirds & Sn...,Coriolanus Snow mentors and develops feelings ...,Francis Lawrence,"Michael Lesslie,Michael Arndt,Suzanne Collins","Action,Adventure,Drama",PG-13,2023.0,157.0,7.2,37.0,100.0,105.043,191.729,11.19.2023,44.607
2,The Killer,"After a fateful near-miss, an assassin battles...",David Fincher,"Andrew Kevin Walker,Luc Jacamon,Alexis Nolent","Action,Adventure,Crime",R,2023.0,118.0,6.8,117.0,NaN,NaN,0.421,NaN,NaN
3,Leo,A 74-year-old lizard named Leo and his turtle ...,"David Wachtenheim,Robert Smigel,Robert Marianetti","Paul Sado,Robert Smigel,Adam Sandler","Animation,Comedy,Family",PG,2023.0,102.0,7.0,10.0,NaN,NaN,NaN,NaN,NaN
4,Thanksgiving,"After a Black Friday riot ends in tragedy, a m...",Eli Roth,"Eli Roth,Jeff Rendell","Horror,Mystery,Thriller",R,2023.0,106.0,7.0,9.1,NaN,25.409,29.667,11.19.2023,10.306


In [5]:
df_extra.columns = df_extra.columns.str.lower().str.replace(' ', '_')
df_extra = df_extra[['title', 'writer', 'main_genres',
       'budget_(in_millions)', 'gross_worldwide_(in_millions)']]

In [6]:
df = df_base.merge(df_extra, on="title", how='inner')

In [7]:
df.shape

(7609, 13)

In [8]:
df.sample(5)

,title,year,certificate,genre,rating,metascore,director,cast,description,writer,main_genres,budget_(in_millions),gross_worldwide_(in_millions)
6755,Trash Humpers,2023.0,NaN,"Comedy, Drama, Horror",4.9,33.0,Harmony Korine,"Rachel Korine, Brian Kotzur, Travis Nicholson,...",Follows the lives of a small group of sociopat...,Harmony Korine,"Comedy,Drama,Horror",NaN,NaN
6473,Sabotage,2022.0,A,"Action, Crime, Drama",5.7,41.0,David Ayer,"Arnold Schwarzenegger, Sam Worthington, Terren...",Members of an elite DEA task force find themse...,"David Ayer,Skip Woods","Action,Crime,Drama",35.0,22.127
7150,Apollo 18,2024.0,PG-13,"Horror, Mystery, Sci-Fi",5.2,24.0,Gonzalo López-Gallego,"Warren Christie, Lloyd Owen, Ryan Robbins, Mic...",Decades-old found footage from NASA's abandone...,"Cory Goodman,Cory Goodman(uncredited),Brian Mi...","Horror,Mystery,Sci-Fi",5.0,26.236
7283,Better Days,2006.0,NaN,Drama,6.6,NaN,Joan Carr-Wiggin,"Sonja Smits, Alix Sideris, Dean Armstrong, Luk...","After Kate's husband dies, she withdraws into ...","Yimeng Xu,Yuan Li,Wing-Sum Lam","Crime,Drama,Romance",NaN,225.919
4068,Risen,2023.0,UA,"Action, Drama, History",6.3,51.0,Kevin Reynolds,"Joseph Fiennes, Tom Felton, Peter Firth, Cliff...","In 33 AD, a Roman Tribune in Judea is tasked t...","Kevin Reynolds,Paul Aiello","Action,Drama,History",20.0,46.772


In [ ]:
def filter_by_year(df, start_year=1990, end_year=2024):
    """
    Filters the dataframe for movies released between start_year and end_year.
    
    Parameters:
    df (DataFrame): The dataframe containing movie data.
    start_year (int): The starting year for the filter.
    end_year (int): The ending year for the filter.
    
    Returns:
    DataFrame: A filtered dataframe containing movies between the specified years.
    """
    # Filter for rows where `year` is between start_year and end_year, inclusive
    filtered_df = df[(df['year'] >= start_year) & (df['year'] <= end_year)]
    return filtered_df

# Example usage:
df = filter_by_year(df)

In [ ]:
# Function to fill missing values in `certificate` with a placeholder
def fill_certificate(df):
    df['certificate'].fillna('Unrated', inplace=True)

# Function to drop rows with missing `year` values
def drop_null_year(df):
    df['year'].dropna(inplace=True)

# Main function to handle all NaN values in df_movies
def handle_missing_values(df):
    fill_certificate(df)
    drop_null_year(df)

handle_missing_values(df)

In [9]:
df['movie_id'] = df.index + 1   # Agregamos un ID único a cada película

# Tabla df_movies
df_movies = df[['movie_id', 'title', 'year', 'certificate', 'rating', 'metascore', 'description', 
                'budget_(in_millions)', 'gross_worldwide_(in_millions)']].copy()

# Tabla df_director (una fila por cada director)
df_director = df[['movie_id', 'director']].copy()
df_director = df_director.assign(director=df_director['director'].str.split(',')).explode('director')
df_director['director'] = df_director['director'].str.strip()

# Tabla df_cast (una fila por cada actor/actriz en el reparto)
df_cast = df[['movie_id', 'cast']].copy()
df_cast = df_cast.assign(cast=df_cast['cast'].str.split(',')).explode('cast')
df_cast.rename(columns={'cast': 'cast_member'}, inplace=True)
df_cast['cast_member'] = df_cast['cast_member'].str.strip()

# Tabla df_writer (una fila por cada guionista)
df_writer = df[['movie_id', 'writer']].copy()
df_writer = df_writer.assign(writer=df_writer['writer'].str.split(',')).explode('writer')
df_writer['writer'] = df_writer['writer'].str.strip()

# Tabla df_genres (una fila por cada género)
df_genres = df[['movie_id', 'main_genres']].copy()
df_genres = df_genres.assign(main_genres=df_genres['main_genres'].str.split(',')).explode('main_genres')
df_genres.rename(columns={'main_genres': 'genre'}, inplace=True)
df_genres['genre'] = df_genres['genre'].str.strip()

In [10]:
all_people = pd.concat([
    df_director['director'],
    df_cast['cast_member'],
    df_writer['writer']])

In [11]:
# Eliminamos duplicados usando unique() y convertimos a una lista
unique_people = all_people.unique().tolist()

# Convertimos a un dataframe para visualizar mejor si lo prefieres
df_persongender = pd.DataFrame(unique_people, columns=['person_name'])

In [14]:
# Initialize the gender detector from `gender-guesser`
d = gender.Detector()

# Function to predict gender using `gender-guesser`
def guess_gender_with_library(name):
    if isinstance(name, str):  # Check if the name is a valid string
        first_name = name.split()[0]  # Use only the first name
        gender_guess = d.get_gender(first_name)
        
        if gender_guess in ["male", "mostly_male"]:
            return "Male"
        elif gender_guess in ["female", "mostly_female"]:
            return "Female"
    return "Unknown"  # Return "Unknown" for non-string or unidentified names

# Apply `gender-guesser` function to predict gender where possible
df_persongender['gender_guess'] = df_persongender['person_name'].apply(guess_gender_with_library)

# Function to predict gender using Genderize.io API with simple error handling
def guess_gender_with_api(name):
    if isinstance(name, str):
        first_name = name.split()[0]
        try:
            response = requests.get(f"https://api.genderize.io/?name={first_name}", timeout=5)
            if response.status_code == 200:
                data = response.json()
                if 'gender' in data and data['gender']:
                    return data['gender'].capitalize()  # Returns 'Male' or 'Female'
        except requests.exceptions.RequestException:
            pass  # If there's a connection issue, return "Unknown"
    return "Unknown"

# Apply the API-based prediction function only to names with "Unknown" from `gender-guesser`
df_persongender['gender'] = df_persongender.apply(
    lambda row: guess_gender_with_api(row['person_name']) if row['gender_guess'] == "Unknown" else row['gender_guess'],
    axis=1
)

# Drop the intermediate column and save the results
df_persongender.drop(columns=['gender_guess'], inplace=True)
df_persongender.to_csv('../data/persongender.csv', index=False)

In [15]:
# Function to print dataframe details: first few rows and data types
def print_df_info(df, name):
    print(f"\nDataframe: {name}")
    print("First few rows:")
    print(df.head())
    print("\nData types:")
    print(df.dtypes)
    print("-" * 40)

# Assuming these dataframes are already defined
# Replace with your actual data if needed
dfs = {
    "df_movies": df_movies,
    "df_director": df_director,
    "df_cast": df_cast,
    "df_writer": df_writer,
    "df_genres": df_genres,
    "df_persongender": df_persongender,
}

# Loop through each dataframe and print details
for name, df in dfs.items():
    print_df_info(df, name)


Dataframe: df_movies
First few rows:
   movie_id                              title    year certificate  rating  \
0         1                    The Idea of You  2023.0           R     6.4   
1         2  Kingdom of the Planet of the Apes  2023.0       PG-13     7.3   
2         3                       The Fall Guy  2023.0       PG-13     7.3   
3         4                        Challengers  2023.0           R     7.7   
4         5                           Twisters  2023.0         NaN     NaN   

   metascore                                        description  \
0       67.0  Solène, a 40-year-old single mom, begins an un...   
1       66.0  Many years after the reign of Caesar, a young ...   
2       73.0  A down-and-out stuntman must find the missing ...   
3       82.0  Tashi, a former tennis prodigy turned coach, t...   
4        NaN  An update to the 1996 film 'Twister', which ce...   

   budget_(in_millions)  gross_worldwide_(in_millions)  
0                   NaN          